In [ ]:
import sys
import os

sys.path.append("..")

from voxeltorch import TSDF, tsdf2meshes


In [ ]:
import torch
from pytorch3d.io import load_obj
from pytorch3d.ops import sample_farthest_points, sample_points_from_meshes, knn_points
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.vis.plotly_vis import plot_scene

def visualize_meshes(meshes:Meshes, title="Mesh"):
    mesh_dict = { }
    
    for idx in range(meshes.__len__()):
        mesh_dict[title + f" {idx}"] = {
            "mesh": meshes[idx]
        }
    fig = plot_scene(mesh_dict, ncols=2)
    fig.update_layout(height=400, width=800)
    return fig.show()

# def visualize_pointcloud(point_cloud:torch.Tensor, title="Point cloud"):
#     point_cloud = Pointclouds(points=point_cloud)
#     fig = plot_scene({
#         title + "1": {
#             "mesh": point_cloud[0]
#         },
#         title + "2": {
#             "mesh": point_cloud[1]
#         },
#     }, ncols=2)
#     fig.update_layout(height=400, width=800)
#     return fig.show()

In [ ]:
mesh1_path = "..\\assets\\models\\benchmark\\019-SUV.obj"
mesh2_path = "..\\assets\\models\\benchmark\\036-CAR01.obj"
verts1, faces1, aux1 = load_obj(mesh1_path, load_textures=False)
verts2, faces2, aux2 = load_obj(mesh2_path, load_textures=False)
bbox = torch.concat([verts1, verts2]).amax(dim=(0)) - torch.concat([verts1, verts2]).amin(dim=(0))
antisotropic_res = (bbox * 10.0).ceil().int()
print(f"bbox: {bbox}")
print(f"resolution: {antisotropic_res}")

orig_mesh = Meshes(verts=[verts1, verts2], faces=[faces1.verts_idx, faces2.verts_idx])
orig_mesh.offset_verts_(torch.Tensor([0.0, -bbox[1].item()/2, 0.0]))
print(orig_mesh.__len__())

tsdf = TSDF(resolution=antisotropic_res, sampling_count=4096, downsampling_count=2048, bbox=bbox, isotropic=True)

tsdf_grid = tsdf.tsdf(orig_mesh)
print(tsdf_grid.size())

In [ ]:
print(orig_mesh.__len__())
print("显示原始模型...")
visualize_meshes(orig_mesh, "Original Mesh")
print("显示体素模型...")
tsdf_meshes = tsdf2meshes(tsdf_grid, antisotropic_res / bbox)
visualize_meshes(tsdf_meshes, "TSDF Mesh")
